In [221]:
import pandas as pd

In [222]:
def read_data(id):
    return pd.read_csv('https://drive.google.com/uc?export=download&id='+id_dict[id])

id_dict = {
    'products': '1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU',
    'orders': '1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ',
    'orderlines': '1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG',
    'brands': '1m1ThDDIYRTTii-rqM5SEQjJ8McidJskD',
}

#org_products_df = read_data('products')
#org_orders_df = read_data('orders')
#org_orderlines_df = read_data('orderlines')
#org_brands_df = read_data('brands')


In [223]:
products_df = org_products_df.copy()
orders_df = org_orders_df.copy()
orderlines_df = org_orderlines_df.copy()
brands_df = org_brands_df.copy()


Cleaing products_df

Duplicates

In [224]:
products_df[products_df.duplicated()].sum()
print("Before:")
products_df.info()
products_df = products_df.drop_duplicates()
print("\n\After:")
products_df.info()

Before:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB

\After:
<class 'pandas.core.frame.DataFrame'>
Index: 10580 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10580 non-null  object
 1   name         10580 non-null  object
 2   desc         10573 non-null  object
 3   price        10534 non-null  object
 4   promo_price  10580 non-null  object
 5   in_stock     10580 non-null  int64 
 6   type         10530 n

In [225]:

products_df[products_df.duplicated()]

,sku,name,desc,price,promo_price,in_stock,type


Assumption: 
Promo prices in the form x.xxx.xxx should be xxxx.xxx -> correcable
Promo prices in the form of xxx.xxx are wrong by a factor of 10. --> corretable
after that : convert to float!

In [226]:
products_df['promo_price'] = products_df['promo_price'].str.replace(r'^(\d*)\.(\d\d\d)\.(\d\d\d)',r'\1\2.\3',regex=True)
products_df.loc[products_df['promo_price'].str.contains(r'^\d*\.\d\d\d$'),'promo_price'] =  products_df.loc[products_df['promo_price'].str.contains(r'^\d*\.\d\d\d$'),'promo_price'].astype(float) / 10
products_df['promo_price'] = pd.to_numeric(products_df['promo_price'])

prices in the form x.xxx.xx should be xxxx.xx -> correcable
Some unitprices are missing (promotional products with onlaa promo price?) --> use promoprice for those
after that : convert to float!

In [227]:
products_df.loc[~products_df['price'].isna()]

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,49.9899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,58.9996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,56.9898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,22.9997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.9900,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,26.9903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,64.9903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,64.9903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,64.9903,0,1392


In [233]:
products_df.loc[~products_df['price'].isna(),'price'] = products_df.loc[~products_df['price'].isna(), 'price'].str.replace(r'^(\d*)\.(\d\d\d)\.(\d\d)$',r'\1\2.\3',regex=True)
products_df.loc[~products_df['price'].isna(),'price'] = products_df.loc[~products_df['price'].isna(), 'price'].str.replace(r'^(\d*)\.(\d\d\d)\.(\d\d\d)$',r'\1\2.\3',regex=True)
products_df.loc[~products_df['price'].isna() & products_df['price'].str.contains(r'^\d*\.\d\d\d$'),'price'] =  products_df.loc[~products_df['price'].isna() & products_df['price'].str.contains(r'^\d*\.\d\d\d$'),'price'].astype(float) / 10

products_df.loc[products_df['price'].isna(),'price'] = products_df.loc[products_df['price'].isna(),'promo_price']
products_df['price'] = pd.to_numeric(products_df['price'])

Just checkin: 
- how many of all productsa are use in orderlines?
- how many rows in orderlines use products, that don`t exist (in products) 

In [ ]:
print(products_df['sku'].nunique())
print(orderlines_df['sku'].nunique())
prol_df = orderlines_df.merge(products_df,on='sku',how='left')
print("There are {:d} rows in orderlines that use products that don't exist in products.\nThose rows use {:d} distinct products:".format(prol_df[prol_df['name'].isna()]['id'].count(), prol_df.loc[prol_df['name'].isna(),'sku'].nunique() ))
prol_df.loc[prol_df['name'].isna(),'sku'].unique()

10579
7951


There are 1418 rows in orderlines that use products that don't exist in products.
Those rows use 101 distinct products:


array(['SYN0127', 'EVU0007', 'APP0608', 'APP0607', 'APP0616', 'PAC0364',
       'APP0615', 'PAC1155', 'REP0255', 'LIF0102', 'MYF0007', 'SAT0040',
       'PAC1773', 'MYF0010', 'RAI0021', 'BLL0010', 'PAC1156', 'PAC1908',
       'PAC1158', 'REP0259', 'CRU0045-A', 'REP0217', 'WAC0131',
       'SYN0133-A', 'PAC1153', 'REP0121', 'WDT0177-A', 'WAC0130',
       'ADO0077', 'IHE0013', 'REP0183', 'REP0228', 'MOX0012', 'FCM0022-A',
       'PAC1157', 'PAC1154', 'BEL0166', 'PAC1910', 'LIBRO', 'PAC1152',
       'PAC1929', 'LEE0015', 'APP0297', 'PAC0469', 'AP20095', 'AP20153',
       'PAC2160', 'ENV0496', 'ENV0505', 'PAC1776', 'AP20172', 'AP20118',
       'PAC0753', 'PAC2165', 'PAC2162', 'PAC2164', 'PAC2166', 'PAC2167',
       'PAC2163', 'AP20220', 'TUC0301', 'NTE0049-A', 'AP20235', 'PIE0081',
       'ENV0805', 'APP2447', 'CAD0011', 'CAD0007', 'APP2448', 'APP2449',
       'CAD0012', 'CAD0010', 'CAD0008', 'LEE0017', 'ENV0494', 'PAC1911',
       'TWS0053-A', 'APP2110', 'AP20301', 'AP20349', 'AP20009',
 

(drop them?)

Get rid of products, that are not referenced in orderlines

In [ ]:
# Keep only products that are referenced in orderlines
products_df = products_df[products_df['sku'].isin(orderlines_df['sku'])]
print(f"Products after filtering: {len(products_df)}")

Products after filtering: 7851


One duplicate sku in products! 

In [ ]:
#rderlines_df['unit_price'] = orderlines_df['unit_price'].str.replace(r'^(\d*)\.(\d\d\d)\.(\d\d)',r'\1\2.\3',regex=True).astype(float)
products_df[products_df['desc'].isna()]

,sku,name,desc,price,promo_price,in_stock,type
16126,WDT0211-A,"Open - Purple 2TB WD 35 ""PC Security Mac hard ...",NaN,107.000,81.4659,0,1298
16128,APP1622-A,Open - Apple Smart Keyboard Pro Keyboard Folio...,NaN,1568.206,156.8206,0,1298
17843,PAC2334,Synology DS718 + NAS Server | 10GB RAM,NaN,566.350,565.9896,0,12175397
18490,HTE0025,Hyper Pearl 1600mAh battery Mini USB Mirror an...,NaN,24.990,22.9900,1,1515
18612,OTT0200,OtterBox External Battery Power Pack 20000 mAHr,NaN,79.990,56.9900,1,1515
18690,HOW0001-A,Open - Honeywell thermostat Lyric zonificador ...,NaN,199.990,144.1174,0,11905404


In [ ]:
#orderlines_df.merge(products_df, on="sku").sort_values("unit_price",ascending=False)

In [ ]:
#orders_df['created_date'] = orders_df['created_date'].astype()